In [1]:
import mlflow
import mlflow.tensorflow
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping


Настройка MLflow

In [2]:
data_name = "ex5_X_all_for_y_2,3,4"
model_name = "LSTM_"
run_name =  model_name + data_name + "_v.1"

# установка пути к хранилищу Mlflow
mlflow.set_tracking_uri("file:///E:/Документы/Data Science/Projects/Equipment-Failure-Prediction/mlruns")
mlflow.set_experiment("Equipment-Failure-Prediction_M3")

def run_mlflow():
    '''Function making one run in current experiment'''

    mlflow.start_run(run_name=run_name)
    mlflow.log_param("X_columns", "X_all_columns")
    mlflow.log_param("y_columns", y.columns)
    mlflow.log_param("split_date", date)
    mlflow.log_param("shuffle_in_model.fit", shuffle)


In [3]:
#mlflow.end_run()
#mlflow.active_run()

In [4]:
#ЭКСГАУСТЕР №5
X = pd.read_parquet('../../data/processed/ex5_M3_X_all_for_y_2,3,4.parquet')
y = pd.read_parquet('../../data/processed/ex5_M3_y_2,3,4.parquet')

In [5]:
X.head(3)

,ЭКСГАУСТЕР 5. ТОК РОТОРА 1,ЭКСГАУСТЕР 5. ТОК РОТОРА 2,ЭКСГАУСТЕР 5. ТОК СТАТОРА,ЭКСГАУСТЕР 5. ДАВЛЕНИЕ МАСЛА В СИСТЕМЕ,ЭКСГАУСТЕР 5. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 1,ЭКСГАУСТЕР 5. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 2,ЭКСГАУСТЕР 5. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 3,ЭКСГАУСТЕР 5. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 4,ЭКСГАУСТЕР 5. ТЕМПЕРАТУРА МАСЛА В СИСТЕМЕ,ЭКСГАУСТЕР 5. ТЕМПЕРАТУРА МАСЛА В МАСЛОБЛОКЕ,ЭКСГАУСТЕР 5. ВИБРАЦИЯ НА ОПОРЕ 1,ЭКСГАУСТЕР 5. ВИБРАЦИЯ НА ОПОРЕ 2,ЭКСГАУСТЕР 5. ВИБРАЦИЯ НА ОПОРЕ 3,ЭКСГАУСТЕР 5. ВИБРАЦИЯ НА ОПОРЕ 3. ПРОДОЛЬНАЯ.,ЭКСГАУСТЕР 5. ВИБРАЦИЯ НА ОПОРЕ 4,ЭКСГАУСТЕР 5. ВИБРАЦИЯ НА ОПОРЕ 4. ПРОДОЛЬНАЯ.
DT,,,,,,,,,,,,,,,,
2019-01-16 13:21:00,1.085000,1.085000,110.895000,126.045000,28.390000,33.260000,34.070000,37.320000,26.770000,26.770000,0.030000,0.030000,0.060000,0.100000,0.050000,0.110000
2019-01-16 13:21:10,1.026476,1.026476,110.894090,126.346667,28.637719,33.180822,34.073210,37.399398,26.690681,25.960000,0.038194,0.043403,0.070313,0.093750,0.078125,0.112847
2019-01-16 13:21:20,1.036844,1.036844,110.860334,126.144540,28.394340,32.991527,34.253491,37.318270,26.771809,26.681666,0.032793,0.039545,0.072338,0.089699,0.064622,0.116705


In [6]:
y.columns

Index(['Y_ЭКСГАУСТЕР А/М №5_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №5',
       'Y_ЭКСГАУСТЕР А/М №5_ЗАДВИЖКА ЭКСГ. №5',
       'Y_ЭКСГАУСТЕР А/М №5_КОРПУС ЭКСГ. №5'],
      dtype='object')

In [7]:
print(X.shape)
print(y.shape)

(7809631, 16)
(7809631, 3)


In [8]:
# Масштабирование  и разделение
scaler = MinMaxScaler()
X_norm = scaler.fit_transform(X)

date = '2020-09-01 08:00:00'
split_date = X.index.get_loc(date)

X_train = X_norm[:split_date]
y_train = y[:split_date]
X_val = X_norm[split_date:]
y_val = y[split_date:]


In [10]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(5086168, 16)
(5086168, 3)
(2723463, 16)
(2723463, 3)


In [11]:
# reshape for input 
timesteps = 1
samples_train = int(np.floor(X_train.shape[0]/timesteps))
samples_val = int(np.floor(X_val.shape[0]/timesteps))

X_train = X_train.reshape(samples_train, timesteps, X_train.shape[1])   #samples, timesteps, sensors
X_val = X_val.reshape(samples_val, timesteps, X_val.shape[1])

print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(5086168, 1, 16)
(5086168, 3)
(2723463, 1, 16)
(2723463, 3)


In [12]:
del X, X_norm

In [13]:
out = y_train.shape[1]
shuffle = False
batch_size = 32
early_stop = EarlyStopping(monitor='val_loss', patience=5)

In [14]:
tf.keras.backend.clear_session()  # очищаем предыдушие следы модели

In [15]:
# Модель
model = tf.keras.Sequential([
    layers.LSTM(64, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    layers.LSTM(64, activation='relu'),
    #layers.Dropout(0.2),
    #layers.Dense(64),
    layers.Dense(out, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 64)             20736     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 3)                 195       
                                                                 
Total params: 53,955
Trainable params: 53,955
Non-trainable params: 0
_________________________________________________________________


In [16]:
run_mlflow()

# Обучение на объединенной выборке
history = model.fit(X_train, y_train,
                    epochs=10,
                    #batch_size=batch_size,
                    shuffle=shuffle,  #False
                    validation_data=(X_val, y_val),
                    callbacks=[early_stop])

Epoch 1/10
158943/158943 [==============================] - 518s 3ms/step - loss: 0.0208 - accuracy: 0.9899 - val_loss: 5.5950 - val_accuracy: 0.9839
Epoch 2/10
158943/158943 [==============================] - 519s 3ms/step - loss: 0.0275 - accuracy: 0.9962 - val_loss: 15.3448 - val_accuracy: 0.9839
Epoch 3/10
158943/158943 [==============================] - 520s 3ms/step - loss: 0.0144 - accuracy: 0.8967 - val_loss: 18.6017 - val_accuracy: 0.9839
Epoch 4/10
158943/158943 [==============================] - 527s 3ms/step - loss: 0.0234 - accuracy: 1.0000 - val_loss: 9.7568 - val_accuracy: 0.9839
Epoch 5/10
158943/158943 [==============================] - 531s 3ms/step - loss: 0.0248 - accuracy: 1.0000 - val_loss: 14.4260 - val_accuracy: 0.9839
Epoch 6/10
158943/158943 [==============================] - 532s 3ms/step - loss: 0.0355 - accuracy: 1.0000 - val_loss: 7.9074 - val_accuracy: 0.9839


Результаты прогноза

In [17]:
y_pred = model.predict(X_val)
y_pred = np.round(y_pred)

85109/85109 [==============================] - 115s 1ms/step


In [18]:
accuracy = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred, average='weighted')

TP = ((y_pred == 1) * (y_val == 1)).sum()
FP = ((y_pred == 1) * (y_val != 1)).sum()
FN = ((y_pred != 1) * (y_val == 1)).sum()
J = TP / (TP + FP + FN)

print('accuracy:', accuracy)
print('f1-score:', f1)
print('J-index', J)

accuracy: 0.18063766608909318
f1-score: 0.6779776124013905
J-index Y_ЭКСГАУСТЕР А/М №5_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №5    0.751285
Y_ЭКСГАУСТЕР А/М №5_ЗАДВИЖКА ЭКСГ. №5                          0.292904
Y_ЭКСГАУСТЕР А/М №5_КОРПУС ЭКСГ. №5                            0.252876
dtype: float64


In [19]:
mlflow.log_metric("accuracy", accuracy)
mlflow.log_metric("f1-score", f1)
mlflow.log_metric("j-index", J.mean())
#mlflow.tensorflow.log_model(model, "model")
mlflow.end_run()